Revision History: Changed the way Punctuations are handled

### Importing all the required Modules

In [1]:
!pip install tensorflow==2.1
!pip install nltk
!pip install tika
!pip install textblob
!pip3 install --upgrade numpy
!pip install scikit-learn

/usr/bin/sh: pip3: command not found


In [2]:
# To handle Paths
import os

# To remove Hyperlinks and Dates
import re

# To remove Puncutations
import string

'''
# For Tokenization and Accessing Stopwords
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('punkt')
'''

# This helps to remove the unnecessary words from our Text Data
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

# To Parse the Input Data Files
from tika import parser

from textblob import TextBlob

# In order to use the Libraries of Tensorflow
import tensorflow as tf

# For Preprocessing the Text => To Tokenize the Text
from tensorflow.keras.preprocessing.text import Tokenizer
# If the Two Articles are of different length, pad_sequences will make the length equal
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Package for performing Numerical Operations
import numpy as np

# MatplotLib for Plotting Graphs
import matplotlib.pyplot as plt

# To shuffle the Data
from random import shuffle

# To Partition the Data into Train Data and Test Data
from sklearn.model_selection import train_test_split

# To add Regularizer in order to reduce Overfitting
from tensorflow.keras.regularizers import l2

In [3]:
tf.__version__

'2.1.0'

In [9]:
# Give the Path of our Data
Path_Of_Data = '/home/ravikrishnak/Desktop/REsume classification/Test'

Labels_List = ['Python', 'Mechanical', 'SAP', 'Testing', 'Networking', 'Civil']

In [10]:
def Pre_Process_Data_And_Create_BOW(folder_path):
  #creating empty lists in order to Create Resume Text and the respective Label
  Resumes_List = []
  Input_Files = []
  for EachLabel in Labels_List:
    for root, dirs, files in os.walk(os.path.join(folder_path, EachLabel),topdown=False):
      if len(files)>= 0:
          for file in files:
            i = 0
            if file.endswith('.pdf'):
              #Access individual file
              Full_Resume_Path = os.path.join(root, file)
              # Parse the Data inside the file
              file_data = parser.from_file(Full_Resume_Path)
              # Extract the Content of the File
              Resume_Text = file_data['content']

              # Below Code removes the Hyperlinks in the Resume, like LinkedIn Profile, Certifications, etc..
              HyperLink_Regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
              Text_Without_HL = re.sub(HyperLink_Regex, ' ', Resume_Text, flags=re.MULTILINE)

              # Below Code removes the Date from the Resume
              Date_regEx = r'(?:\d{1,2}[-/th|st|nd|rd\s]*)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)?[a-z\s,.]*(?:\d{1,2}[-/th|st|nd|rd)\s,]*)+(?:\d{2,4})+'
              CleanedText = re.sub(Date_regEx,' ',Text_Without_HL)

              List_Of_All_Punctuations = list(string.punctuation)
              Important_Punctuations = ['#', '.', '+' , '-'] #Add more, if any other Punctuation is observed as Important

              NewLineChar = '\n'

              # Below Set Comprises all the Punctuations, which can be Removed from the Text of Resume
              Total_Punct = len(List_Of_All_Punctuations)

              for EachImpPunct in Important_Punctuations:
                  for CountOfPunct in range(Total_Punct):
                      if CountOfPunct == Total_Punct:
                          break
                      elif EachImpPunct == List_Of_All_Punctuations[CountOfPunct]:
                          del List_Of_All_Punctuations[CountOfPunct]
                          Total_Punct = Total_Punct - 1

              List_Of_All_Punctuations.append(NewLineChar)

              for EachPunct in List_Of_All_Punctuations:
                  CleanedText = CleanedText.replace(EachPunct, " ")

              # Below Code converts all the Words in the Resume to Lowercase ======> Check if it has to come after Tokenization if Splitting Code is delet instead of integed
              #Final_Cleaned_Resume_Text = Text_Without_Punct.lower()
              Final_Cleaned_Resume_Text = CleanedText.lower()

              #Code to remove Stopwords from each Resume
              for word in STOPWORDS:            
                  stop_token = word
                  Resume_Text = Final_Cleaned_Resume_Text.replace(stop_token, ' ')            
              Resumes_List.append(Resume_Text)
              Input_Files.append(EachLabel + '_' + file)
  return Input_Files, Resumes_List

In [11]:
#calling the function and passing the path
Input_Files, Resumes_List = Pre_Process_Data_And_Create_BOW(Path_Of_Data)

In [12]:
print('Input_Files = {}'.format(Input_Files))

Input_Files = ['Python_9.pdf', 'Python_13.pdf', 'Python_6.pdf', 'Python_5.pdf', 'Python_3.pdf', 'Python_8.pdf', 'Python_12.pdf', 'Python_2.pdf', 'Python_10.pdf', 'Python_11.pdf', 'Python_1.pdf', 'Python_14.pdf', 'Python_7.pdf', 'Python_4.pdf', 'Mechanical_9.pdf', 'Mechanical_13.pdf', 'Mechanical_6.pdf', 'Mechanical_5.pdf', 'Mechanical_3.pdf', 'Mechanical_8.pdf', 'Mechanical_12.pdf', 'Mechanical_2.pdf', 'Mechanical_10.pdf', 'Mechanical_11.pdf', 'Mechanical_1.pdf', 'Mechanical_14.pdf', 'Mechanical_7.pdf', 'Mechanical_4.pdf', 'SAP_6(1).pdf', 'SAP_6.pdf', 'SAP_5.pdf', 'SAP_3.pdf', 'SAP_12.pdf', 'SAP_2.pdf', 'SAP_17.pdf', 'SAP_10.pdf', 'SAP_20.pdf', 'SAP_19.pdf', 'SAP_16.pdf', 'SAP_1.pdf', 'SAP_18.pdf', 'SAP_16(1).pdf', 'Testing_9.pdf', 'Testing_13.pdf', 'Testing_6.pdf', 'Testing_5.pdf', 'Testing_3.pdf', 'Testing_8.pdf', 'Testing_12.pdf', 'Testing_2.pdf', 'Testing_10.pdf', 'Testing_11.pdf', 'Testing_1.pdf', 'Testing_14.pdf', 'Testing_7.pdf', 'Testing_4.pdf', 'Networking_9.pdf', 'Networking_

In [13]:
vocab_size = 5000
# We want the Output of the Embedding Layer to be 64
embedding_dim = 64
#max_length = 200
max_length = 800
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(Resumes_List)
word_index = tokenizer.word_index

# Convert the Word Tokens into Integer equivalents, before passing it to keras embedding layer
Predict_Sequences = tokenizer.texts_to_sequences(Resumes_List)

Predict_Padded = pad_sequences(Predict_Sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [14]:
# Contains an assets folder, saved_model.pb, and variables folder.
!ls Resume_Classification_Model

1


In [15]:
New_Model = tf.keras.models.load_model("Resume_Classification_Model/1")

New_Model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          320000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 128)         66048     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 390       
Total params: 431,814
Trainable params: 431,814
Non-trainable params: 0
_________________________________________________________________


In [16]:
Prediction = New_Model.predict(Predict_Padded)

Prediction_Classes = tf.argmax(Prediction, axis=1)

for Input_File_Name, Pred_Class in zip(Input_Files, Prediction_Classes):
    print('Prediction for {} is {}'.format(Input_File_Name, Labels_List[(Pred_Class.numpy()-1)]))

Prediction for Python_9.pdf is Civil
Prediction for Python_13.pdf is Networking
Prediction for Python_6.pdf is Networking
Prediction for Python_5.pdf is Networking
Prediction for Python_3.pdf is Networking
Prediction for Python_8.pdf is Networking
Prediction for Python_12.pdf is Networking
Prediction for Python_2.pdf is Networking
Prediction for Python_10.pdf is Networking
Prediction for Python_11.pdf is Networking
Prediction for Python_1.pdf is Networking
Prediction for Python_14.pdf is Networking
Prediction for Python_7.pdf is Networking
Prediction for Python_4.pdf is Networking
Prediction for Mechanical_9.pdf is Networking
Prediction for Mechanical_13.pdf is Networking
Prediction for Mechanical_6.pdf is Networking
Prediction for Mechanical_5.pdf is Networking
Prediction for Mechanical_3.pdf is Networking
Prediction for Mechanical_8.pdf is Networking
Prediction for Mechanical_12.pdf is Networking
Prediction for Mechanical_2.pdf is Networking
Prediction for Mechanical_10.pdf is Networ